In [1]:
%load_ext autoreload
%autoreload 2

import geort
import time
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

from manus.manus_data_mqtt import GloveMQTTSubscriber

In [2]:
class manus_geort_mocap:
    def __init__(self):
        self._manus_client = GloveMQTTSubscriber()
        time.sleep(1)
        # self._ data_indices = [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
        
    def get(self):
        data_temp = np.array(self._manus_client._right_data["positions"])
        data = np.zeros_like(data_temp)
        data[:,0] = -data_temp[:,1]
        data[:,1] = -data_temp[:,0]
        data[:,2] = +data_temp[:,2]
        return data

In [3]:
# Your data collection loop.
mocap = manus_geort_mocap() # TODO(): your mocap system.
                           # Define a mocap.get() method.
                           # Apologies, you still have to do this...

**** MQTT INFO ****
Server IP (port): localhost (1883)
connected OK


# Data collection

In [4]:
# Dataset Name
data_output_name = "human_LYS_right" # TODO(): Specify a name for this (e.g. your name)

data = []

data_len = 5000
for step in range(data_len):       # collect 5000 data points.
    hand_keypoint = mocap.get() # mocap.get() return [N, 3] numpy array.
    data.append(hand_keypoint)
    
    time.sleep(0.01)            # take a short break.
    if (step+1)%100==0:
        clear_output(wait=True)
        print(f'{(step+1)/data_len * 100} %')

# finish data collection.
geort.save_human_data(data, data_output_name)

32.0 %


KeyboardInterrupt: 

# Train

In [5]:
# !cd ../ && python ./geort/trainer.py -hand kistar_right -human_data human_LYS_right -ckpt_tag geort_LYS_right
!python ./geort/trainer.py -hand kistar_right -human_data human_LYS_right -ckpt_tag geort_LYS_right_kistar --w_pinch 50 --w_curvature 0.1
# !python ../geort/trainer.py -hand allegro_right -human_data human_LYS_right -ckpt_tag geort_LYS_right_allegro

User-to-Sim Joint [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Sim-to-User Joint [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
[ 0.       -1.5708    0.        0.       -0.261799  0.        0.
  0.       -0.261799  0.        0.        0.       -0.08      0.
  0.        0.      ] [1.5708   0.15708  1.5708   1.5708   0.08     1.5708   1.5708   1.5708
 0.261799 1.5708   1.5708   1.5708   0.261799 1.5708   1.5708   1.5708  ]
0 thumb_joint_0 <sapien.core.pysapien.Joint object at 0x77b924f768f0> 0.0 1.5708
1 thumb_joint_1 <sapien.core.pysapien.Joint object at 0x77b924f76930> -1.5708 0.15708
2 thumb_joint_2 <sapien.core.pysapien.Joint object at 0x77b924f76970> 0.0 1.5708
3 thumb_joint_3 <sapien.core.pysapien.Joint object at 0x77b924f769b0> 0.0 1.5708
4 index_joint_0 <sapien.core.pysapien.Joint object at 0x77b924f769f0> -0.261799 0.08
5 index_joint_1 <sapien.core.pysapien.Joint object at 0x77b924f76a30> 0.0 1.5708
6 index_joint_2 <sapien.core.pysapien.Joint object at 0x77b924

In [ ]:
# !cd ../ && python ./geort/mocap/replay_evaluation.py -hand kistar_right -ckpt_tag geort_LYS_right -data human_LYS_right
!python ./geort/mocap/replay_evaluation.py -hand kistar_right -ckpt_tag geort_LYS_right_kistar -data human_LYS_right

# Run

In [6]:
from geort.env.hand import HandKinematicModel

In [7]:
checkpoint_tag = 'geort_LYS_right_kistar'          # TODO: your checkpoint name, assume it is 'TAG'
model = geort.load_model(checkpoint_tag, epoch=-1)  # set epoch=-1 to use the last model.

# Load Hand Model
hand_name = 'kistar_right'
config = geort.utils.config_utils.get_config(hand_name)
hand_model = HandKinematicModel.build_from_config(config, render=True)
viewer_env = hand_model.get_viewer_env()

steps = 0
while True:
    viewer_env.update()

    steps += 1
    qpos = model.forward(mocap.get()) # This is the retargeted qpos. 
                                      # (Note: unnormalized joint angle)
    hand_model.set_qpos_target(qpos)

[[4, 5, 6, 7], [8, 9, 10, 11], [12, 13, 14, 15], [0, 1, 2, 3]]


/home/rnbmarch/Desktop/LYS/GeoRT/geort/export.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(model_path))
/home/rnbmarch/anaconda

Enable Render Mode.
User-to-Sim Joint [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Sim-to-User Joint [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
[ 0.       -1.5708    0.        0.       -0.261799  0.        0.
  0.       -0.261799  0.        0.        0.       -0.08      0.
  0.        0.      ] [1.5708   0.15708  1.5708   1.5708   0.08     1.5708   1.5708   1.5708
 0.261799 1.5708   1.5708   1.5708   0.261799 1.5708   1.5708   1.5708  ]
0 thumb_joint_0 <sapien.core.pysapien.Joint object at 0x73799d1274f0> 0.0 1.5708
1 thumb_joint_1 <sapien.core.pysapien.Joint object at 0x7379950fde70> -1.5708 0.15708
2 thumb_joint_2 <sapien.core.pysapien.Joint object at 0x7379950fdf30> 0.0 1.5708
3 thumb_joint_3 <sapien.core.pysapien.Joint object at 0x7379950fdff0> 0.0 1.5708
4 index_joint_0 <sapien.core.pysapien.Joint object at 0x7379950fddb0> -0.261799 0.08
5 index_joint_1 <sapien.core.pysapien.Joint object at 0x7379950fde30> 0.0 1.5708
6 index_joint_2 <sapien.core.pysapien.Join

KeyboardInterrupt: 

In [ ]:
data_plot = np.array(np.load("./data/human_LYS_right.npy"))

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(projection='3d')

ax.scatter(data_plot[:,4,0], data_plot[:,4,1], data_plot[:,4,2], s=0.1, c='k', alpha=0.5)
ax.scatter(data_plot[:,9,0], data_plot[:,9,1], data_plot[:,9,2], s=0.1, c='r', alpha=0.5)
ax.scatter(data_plot[:,14,0], data_plot[:,14,1], data_plot[:,14,2], s=0.1, c='g', alpha=0.5)
ax.scatter(data_plot[:,19,0], data_plot[:,19,1], data_plot[:,19,2], s=0.1, c='b', alpha=0.5)

ax.axis('equal')
ax.view_init(elev=15, azim=45) 
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()

In [ ]:
from geort.trainer import GeoRTTrainer

hand_name = 'kistar_right'
config = geort.utils.config_utils.get_config(hand_name)

trainer = GeoRTTrainer(config)
fk_dataset = trainer.get_robot_kinematics_dataset()

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(projection='3d')

thumb_data = fk_dataset.export_robot_pointcloud(['thumb_tip_end'])[0][::5,:]
index_data = fk_dataset.export_robot_pointcloud(['index_tip_end'])[0][::5,:]
middle_data = fk_dataset.export_robot_pointcloud(['middle_tip_end'])[0][::5,:]
ring_data = fk_dataset.export_robot_pointcloud(['ring_tip_end'])[0][::5,:]

ax.scatter(thumb_data[:,0], thumb_data[:,1], thumb_data[:,2], s=0.1, c='k', alpha=0.5)
ax.scatter(index_data[:,0], index_data[:,1], index_data[:,2], s=0.1, c='r', alpha=0.5)
ax.scatter(middle_data[:,0], middle_data[:,1], middle_data[:,2], s=0.1, c='g', alpha=0.5)
ax.scatter(ring_data[:,0], ring_data[:,1], ring_data[:,2], s=0.1, c='b', alpha=0.5)

ax.axis('equal')
ax.view_init(elev=15, azim=45) 
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()

In [ ]:
import pybullet as p

p.connect(p.GUI)
p.configureDebugVisualizer(p.COV_ENABLE_GUI, 0)
p.configureDebugVisualizer(p.COV_ENABLE_SHADOWS, 1)
p.configureDebugVisualizer(p.COV_ENABLE_RGB_BUFFER_PREVIEW, 0)
p.configureDebugVisualizer(p.COV_ENABLE_DEPTH_BUFFER_PREVIEW, 0)
p.configureDebugVisualizer(p.COV_ENABLE_SEGMENTATION_MARK_PREVIEW, 0)

p.setPhysicsEngineParameter(enableFileCaching=0)
p.resetDebugVisualizerCamera(cameraDistance=0.4, cameraYaw=145, cameraPitch=-30, cameraTargetPosition=[0, 0, 0])

p.loadURDF("./assets/kistar_right/model.urdf")

In [ ]:
p.addUserDebugPoints(pointPositions=thumb_data, pointColorsRGB=[(0,0,0)]*len(thumb_data), pointSize=1, lifeTime=5)
p.addUserDebugPoints(pointPositions=index_data, pointColorsRGB=[(1,0,0)]*len(thumb_data), pointSize=1, lifeTime=5)
p.addUserDebugPoints(pointPositions=middle_data, pointColorsRGB=[(0,1,0)]*len(thumb_data), pointSize=1, lifeTime=5)
p.addUserDebugPoints(pointPositions=ring_data, pointColorsRGB=[(0,0,1)]*len(thumb_data), pointSize=1, lifeTime=5)

In [ ]:
p.disconnect()